<a href="https://colab.research.google.com/github/noelmtv/Colab-Learning/blob/main/Portfolio_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ============================================
# Portfolio Beta and Sharpe Ratio Calculator
# Using yFinance (Google Colab compatible)
# ============================================

import yfinance as yf
import pandas as pd
import numpy as np

# ============================================
# 1. USER INPUTS
# ============================================

# List of stock tickers in the portfolio
tickers = ['AAPL', 'MSFT', 'GOOGL']

# Corresponding portfolio weights (must sum to 1)
weights = np.array([0.4, 0.35, 0.25])

# Market benchmark (S&P 500)
market_ticker = '^GSPC'

# Risk-free rate (annual, decimal form)
# Example: 4% = 0.04
risk_free_rate = 0.04

# Analysis period
start_date = '2020-01-01'
end_date = '2025-01-01'

# Trading days per year (used for annualisation)
TRADING_DAYS = 252


In [5]:

# ============================================
# 2. VALIDATION CHECKS
# ============================================

if len(tickers) != len(weights):
    raise ValueError("Number of tickers must match number of weights.")

if not np.isclose(weights.sum(), 1):
    raise ValueError("Portfolio weights must sum to 1.")




In [6]:
# ============================================
# 3. DOWNLOAD PRICE DATA
# ============================================

# Combine stock tickers and market index
all_tickers = tickers + [market_ticker]

# Download adjusted close prices
prices = yf.download(
    all_tickers,
    start=start_date,
    end=end_date,
    progress=False
)['Close']

# Drop rows with missing values
prices.dropna(inplace=True)



/tmp/ipython-input-1324595104.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(


In [7]:
# ============================================
# 4. CALCULATE DAILY RETURNS
# ============================================

# Compute daily log returns
returns = np.log(prices / prices.shift(1)).dropna()

# Separate asset returns and market returns
asset_returns = returns[tickers]
market_returns = returns[market_ticker]


In [8]:
# ============================================
# 5. CALCULATE BETAS
# ============================================

betas = {}

for ticker in tickers:
    # Covariance between stock and market
    cov = np.cov(asset_returns[ticker], market_returns)[0, 1]

    # Variance of market returns
    market_var = np.var(market_returns)

    # CAPM beta formula
    beta = cov / market_var
    betas[ticker] = beta

# Convert to DataFrame
beta_df = pd.DataFrame.from_dict(betas, orient='index', columns=['Beta'])

# Portfolio beta (weighted average of individual betas)
portfolio_beta = np.dot(weights, beta_df['Beta'])



In [9]:
# ============================================
# 6. PORTFOLIO RETURNS & VOLATILITY
# ============================================

# Daily portfolio returns
portfolio_returns = asset_returns.dot(weights)

# Annualised portfolio return
annual_return = portfolio_returns.mean() * TRADING_DAYS

# Annualised portfolio volatility
annual_volatility = portfolio_returns.std() * np.sqrt(TRADING_DAYS)



In [10]:
# ============================================
# 7. SHARPE RATIO
# ============================================

# Sharpe Ratio formula:
# (Expected Portfolio Return – Risk-Free Rate) / Portfolio Volatility
sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility



In [11]:
# ============================================
# 8. OUTPUT RESULTS
# ============================================

print("Individual Stock Betas")
display(beta_df)

print(f"\nPortfolio Beta: {portfolio_beta:.4f}")
print(f"Annualised Portfolio Return: {annual_return:.2%}")
print(f"Annualised Portfolio Volatility: {annual_volatility:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")

Individual Stock Betas


,Beta
AAPL,1.171235
MSFT,1.172839
GOOGL,1.132286



Portfolio Beta: 1.1621
Annualised Portfolio Return: 22.11%
Annualised Portfolio Volatility: 28.43%
Sharpe Ratio: 0.6370
